# Claims made about poultry, pork, and beef.

### “poultry costs U.S. consumers 62% less in inflation-adjusted terms than it did in 1935”

### “Pork, now also raised mostly at factory scale indoors, is 12% cheaper”

### “Beef, which isn’t, costs 63% more. “